In [4]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.1.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Get:7 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:8 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:11 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:14 http://ppa.launchpad.net/c2d4u.team/

In [5]:
#Download the Postgres driver that will allow Spark to interact with Postgres
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2023-01-22 00:42:04--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  6.27MB/s    in 0.2s    

2023-01-22 00:42:04 (6.27 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [6]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M17-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [7]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Apparel_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Apparel_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   32158956|R1KKOXHNI8MSXU|B01KL6O72Y|      24485154|Easy Tool Stainle...|         Apparel|          4|            0|          0|   N|                Y|★ THESE REALLY DO...|These Really Do W...| 2013-01-14|
|         US|    2714559|R26SP2OPDK4HT7|B01ID3ZS5W|     363128556|V28 Women Cowl Ne...|         Apparel|          5|    

In [8]:
#Recreating vine_table dataframe from Deliverable 1
vine_df = df.select(['review_id', 'star_rating','helpful_votes','total_votes','vine','verified_purchase'])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1KKOXHNI8MSXU|          4|            0|          0|   N|                Y|
|R26SP2OPDK4HT7|          5|            1|          2|   N|                Y|
| RWQEDYAX373I1|          5|            0|          0|   N|                Y|
|R231YI7R4GPF6J|          5|            0|          0|   N|                Y|
|R3KO3W45DD0L1K|          5|            0|          0|   N|                Y|
|R1C4QH63NFL5NJ|          5|            0|          0|   N|                Y|
|R2GP65O1U9N7BP|          5|            0|          0|   N|                Y|
|R3O29CT5MQQ3XQ|          4|            0|          0|   N|                Y|
|R1ZECD2AA8QFF6|          5|            0|          0|   N|                Y|
|R2S79GCF6J89OA|          3|            0|          0|   N|     

In [10]:
# Filter the DataFrame to narrow on helpful_reviews (total_votes >=20)
filtered_df = vine_df.filter ("total_votes>= 20")
filtered_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R35PT06NWP7LDP|          5|           30|         32|   N|                N|
|R2P76PJFUGXBHO|          1|            6|         45|   N|                N|
| RQW4AFOG9MR4Z|          5|           51|         52|   N|                N|
|R2SMUEBMGLAJQK|          5|           29|         36|   N|                Y|
|R1XVIZZALU5P6J|          5|          148|        150|   N|                Y|
|R11UFMG8M2488I|          4|           23|         26|   N|                N|
|R2OSHKSPXU68W5|          5|          136|        147|   N|                N|
|R3M02FSD3BLUPU|          5|           19|         21|   N|                N|
|R1P0LEEJHH09L3|          5|           32|         32|   N|                Y|
|R1V6UU1EHW3Q12|          5|           57|         62|   N|     

In [14]:
from pyspark.sql.types import FloatType
filtered_float_df = filtered_df.withColumn('helpful_votes', filtered_df['helpful_votes'].cast(FloatType()))
filtered_float_df = filtered_df.withColumn('total_votes', filtered_df['total_votes'].cast(FloatType()))

In [15]:
#filter new DataFrame further where the percentage of helpful_votes is equal to or greater than 50% 
helpful_df = filtered_float_df.filter("helpful_votes/total_votes >= .50")
helpful_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R35PT06NWP7LDP|          5|           30|       32.0|   N|                N|
| RQW4AFOG9MR4Z|          5|           51|       52.0|   N|                N|
|R2SMUEBMGLAJQK|          5|           29|       36.0|   N|                Y|
|R1XVIZZALU5P6J|          5|          148|      150.0|   N|                Y|
|R11UFMG8M2488I|          4|           23|       26.0|   N|                N|
|R2OSHKSPXU68W5|          5|          136|      147.0|   N|                N|
|R3M02FSD3BLUPU|          5|           19|       21.0|   N|                N|
|R1P0LEEJHH09L3|          5|           32|       32.0|   N|                Y|
|R1V6UU1EHW3Q12|          5|           57|       62.0|   N|                Y|
|R230W87COFSTCL|          5|           21|       21.0|   N|     

In [18]:
# filter to create a DataFrame or table where there is a Vine review 
y_vine_df = helpful_df.filter(helpful_df.vine == 'Y')
y_vine_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R6U9701C3BGO6|          3|          139|      147.0|   Y|                N|
|R1XK3ALB45D7N4|          5|           33|       34.0|   Y|                N|
|R1IZCSTLX81D6C|          5|           31|       33.0|   Y|                N|
|R2C8NC8EQLH4JF|          3|           45|       48.0|   Y|                N|
|R1JJ1YOJMOML1P|          5|           18|       21.0|   Y|                N|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [20]:
#filtered to create a DataFrame or table where there isn’t a Vine review
n_vine_df = helpful_df.filter(helpful_df.vine == 'N')
n_vine_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R35PT06NWP7LDP|          5|           30|       32.0|   N|                N|
| RQW4AFOG9MR4Z|          5|           51|       52.0|   N|                N|
|R2SMUEBMGLAJQK|          5|           29|       36.0|   N|                Y|
|R1XVIZZALU5P6J|          5|          148|      150.0|   N|                Y|
|R11UFMG8M2488I|          4|           23|       26.0|   N|                N|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [21]:

# Total number of reviews (Paid vs Unpaid)
paid_total = y_vine_df.count()
unpaid_total = n_vine_df.count()

print("Total Number of helpful, PAID Apparel Reviews: %f" % paid_total)
print("Total Number of helpful, UNPAID Apparel Reviews: %f" % unpaid_total)


Total Number of helpful, PAID Apparel Reviews: 33.000000
Total Number of helpful, UNPAID Apparel Reviews: 45388.000000


In [22]:
# Total Number of 5-star reviews (Paid vs Unpaid)
paid_5star = y_vine_df.filter(y_vine_df['star_rating'] == 5)
unpaid_5star = n_vine_df.filter(n_vine_df['star_rating'] == 5)

print("Number of 5-star helpful, PAID Apparel_Reviews: %f" % paid_5star.count())
print("Number of 5-star helpful UNPAID Apparel Reviews: %f" % unpaid_5star.count())

Number of 5-star helpful, PAID Apparel_Reviews: 15.000000
Number of 5-star helpful UNPAID Apparel Reviews: 23733.000000


In [23]:
# Percentage of 5-star reviews (Paid vs Unpaid)
paid_5star = paid_5star.count()
unpaid_5star = unpaid_5star.count()

paid_5star_percentage = (paid_5star/paid_total) * 100
unpaid_5star_percentage = (unpaid_5star/unpaid_total) * 100
print("Percentage of 5-star, PAID Apparel Reviews: %f" % paid_5star_percentage)
print("Number of Helpful, 5-star UNPAID Apparel Reviews: %f" % unpaid_5star_percentage)

Percentage of 5-star, PAID Apparel Reviews: 45.454545
Number of Helpful, 5-star UNPAID Apparel Reviews: 52.289151
